In [ ]:
from sqlalchemy import or_
from flask import Flask, render_template, url_for, request, redirect, session
from flask.helpers import flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user
from flask_bcrypt import Bcrypt
import os,sys

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///todo.db'
app.config['SECRET_KEY'] = 'mysecret'
db = SQLAlchemy(app)
bcrypt = Bcrypt(app)

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_message = "You need to login again."
login_manager.refresh_view = 'login'
login_manager.login_view = 'login'

class User(db.Model, UserMixin):
    __tablename__ = 'user'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    username = db.Column(db.String(50), unique = True)
    password = db.Column(db.String(80))
    email = db.Column(db.String(255))
    role_id = db.Column(db.Integer, db.ForeignKey('role.id'))

    def __repr__(self):
        return '<User %r: %s>' % (self.id, self.username)

class Role(db.Model):
    __tablename__ = 'role'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(50), unique = True)
    description = db.Column(db.String(50))
    users = db.relationship("User", backref = "role")

    def __repr__(self):
        return '<Role %r: %s>' % (self.id, self.name)

class Manufacturer(db.Model):
    __tablename__ = 'manufacturer'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(200), nullable = False)

    def __repr__(self):
        return '<Manufacturer %r: %s>' % (self.id, self.name)

class Product(db.Model):
    __tablename__ = 'product'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(200), nullable = False)
    category_id = db.Column(db.Integer, db.ForeignKey('category.id'))
    category = db.relationship("Category", backref = "products")
    manufacturer_id = db.Column(db.Integer, db.ForeignKey('manufacturer.id'))
    manufacturer = db.relationship("Manufacturer", backref = "products")
    image = db.Column(db.String(200))
    url = db.Column(db.String(200))
    price = db.Column(db.Float)
    size = db.Column(db.String(20))
    type = db.Column(db.String(20))
    tag = db.Column(db.String(20))
    score = db.Column(db.Float)

    def __repr__(self):
        return '<Product %r: %s>' % (self.id, self.name)

class Category(db.Model):
    __tablename__ = 'category'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    name = db.Column(db.String(200), nullable = False)
    friendly_name = db.Column(db.String(200), nullable = False)
    animation_name = db.Column(db.String(200), nullable = False)
    description = db.Column(db.String(200), nullable = False)

    def __repr__(self):
        return '<Category %r: %s>' % (self.id, self.name)

class UserRate(db.Model):
    __tablename__ = 'userrate'
    id = db.Column(db.Integer, primary_key = True, autoincrement = True)
    user_id = db.Column(db.Integer, db.ForeignKey('user.id'))
    user = db.relationship("User", backref = "rates")
    product_id = db.Column(db.Integer, db.ForeignKey('product.id'))
    product = db.relationship("Product", backref = "rates")
    score = db.Column(db.Integer)
    comment = db.Column(db.String(200))

    def __repr__(self):
        return '<UserRate %r: Rate from User %s to Product %s>' % (self.id, self.user, self.product)

def calculate_product_score(product):
    score = 0
    for rate in product.rates:
        score +=  rate.score
    if len(product.rates) > 0:
        score = score / len(product.rates)
    return "%.1f"%score

if not os.path.exists(sys.path[0] + '/todo.db'):
    db.create_all()
    db.session.add(Role(name = "manufacturer", description = "Manufacturer User"))
    db.session.add(Role(name = "user", description = "Regular User"))
    db.session.commit()

@app.route('/signup', methods = ['GET','POST'])
def signup():
    if request.method == "POST":
        username = request.form['username']
        email = request.form['email']
        password = request.form['password']
        hashed_password = bcrypt.generate_password_hash(password)
        regtype = request.form['regtype']
        new_user = User(
            username = username,
            password = hashed_password,
            email = email,
            role_id = Role.query.filter_by(name = regtype).first().id)
        try:
            db.session.add(new_user)
            db.session.commit()
            return redirect(url_for('login'))
        except Exception as e:
            print(e)
            return 'Fail to add user.'
    else:
        regtype = request.args.get("regtype")
        isBrand = regtype == 'brand'
        return render_template('signup.html', isBrand = isBrand)

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))

@app.route('/login', methods = ['GET','POST'])
def login():
    if request.method == "POST":
        user = User.query.filter_by(username = request.form["username"]).first()
        if user:
            if bcrypt.check_password_hash(user.password, request.form["password"]):
                login_user(user)
                if user.role.name == "manufacturer":
                    return redirect("/mypost")
                else:
                    return redirect("/")
    return render_template('login.html')

@app.route('/logout')
@login_required
def logout():
    logout_user()
    return redirect(url_for('login'))

@app.route("/")
def home():
    return render_template('home.html')

@app.route("/mypost")
def mypost():
    return render_template('mypost.html')

@app.route("/product", methods = ['GET', 'POST'])
def product():
    action = request.args.get("action")

    if action == 'list' and request.method == 'GET':
        category = request.args.get("category")
        category = Category.query.filter_by(name=category).first()
        products = Product.query.filter_by(category_id = category.id).order_by(Product.score.desc()).all()
        return render_template("product_list.html", category = category, products = products)
    elif action == 'detail' and request.method == 'GET':
        product_id = request.args.get("product_id")
        product = Product.query.filter_by(id = product_id).first()
        user_id = None
        user_comment = None
        if '_user_id' in session:
            user_id = session['_user_id']
            user_comment = UserRate.query.filter_by(user_id = user_id).filter_by(product_id = product_id).first()
        return render_template('product_detail.html', product = product, user_id = user_id, user_comment = user_comment)
    elif action == 'rate' and request.method == 'POST':
        product_id = request.form.get("product_id")
        user_id = request.form.get("user_id")
        score = request.form.get("score")
        comment = request.form.get("comment")
        commentObj = UserRate.query.filter_by(user_id = user_id).filter_by(product_id = product_id).first()
        if (commentObj !=  None):
            commentObj.score = score
            commentObj.comment = comment
        else:
            db.session.add(UserRate(user_id = user_id, product_id = product_id, score = score, comment = comment))
        db.session.commit()
        return redirect(url_for('product', action = 'detail', product_id = product_id))
    elif action == 'compare' and request.method == 'GET':
        ids = request.args.get("ids")
        ids = ids.split(",")
        if len(ids) == 2:
            products = Product.query.filter(or_(Product.id == ids[0], Product.id == ids[1])).all()
        elif len(ids) == 3:
            products = Product.query.filter(or_(Product.id == ids[0], Product.id == ids[1], Product.id == ids[2])).all()
        elif len(ids) == 4:
            products = Product.query.filter(or_(Product.id == ids[0], Product.id == ids[1], Product.id == ids[2], Product.id == ids[3])).all()
        else:
            return 'Invalid argument for comparing!'
        return render_template('product_compare.html', products = products)
    else:
        return 'Invalid request.'

if __name__ == "__main__":
    app.run('localhost', 9907)


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


/Users/zhe/opt/anaconda3/lib/python3.8/site-packages/flask_sqlalchemy/__init__.py:872: FSADeprecationWarning: SQLALCHEMY_TRACK_MODIFICATIONS adds significant overhead and will be disabled by default in the future.  Set it to True or False to suppress this warning.
  warnings.warn(FSADeprecationWarning(
 * Running on http://localhost:9907/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Dec/2021 11:41:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 11:41:39] "GET /product?action=list&category=shave HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 11:41:40] "GET /product?action=list&category=perfume HTTP/1.1" 200 -
127.0.0.1 - - [26/Dec/2021 11:41:49] "GET /product?action=detail&product_id=10 HTTP/1.1" 200 -
